In [1]:
using PyPlot

INFO: Loading help data...


In [1]:
type Maze
    definition::String
    string_states::Matrix{Char}
    maze::Matrix{Int64}
    values::Vector{Int64}
    policy::Vector{String}
    special_values::Dict{Any, Any}
end

In [2]:
maze_def="
#######
#T    #
#A    #
#     #
#     #
#    T#
#######
"

state_values = {'T' => 0, ' ' => -1}

Dict{Any,Any} with 2 entries:
  ' ' => -1
  'T' => 0

In [11]:
function convert_string_to_string_array(s::String)
    index = 1
    while s[index] == ' ' || s[index] == '\n'
        index = index + 1
    end
    maze = nothing
    row = []
    for i=index:length(s)
        if s[i] == '\n'
            if maze == nothing
                maze = row
            else
                maze = hcat(maze, row)
            end
            row = []
        else
            row = [row, s[i]]
        end
    end
    return maze
end

function name_states(maze::Matrix{Char})
    states = zeros(size(maze))
    state_name = 1
    for i=1:size(maze)[1]
        for j=1:size(maze)[2]
            if maze[i,j] == '#'
                states[i,j] = 0
            elseif maze[i,j] == ' '
                states[i,j] = state_name
                state_name = state_name + 1
            else
                states[i,j] = -1    
            end
        end
    end
    int64(states)
end

initialise_value_function(maze::Matrix{Int64}) = zeros(maximum(maze))

function greedy_policy!(m::Maze)
    maze   = m.maze
    values = m.values
    m.policy = ["" for i=1:length(values)]
    for i = 2:size(maze)[1]-1
        for j = 2:size(maze)[2]-1
            if maze[i,j] > 0
                possible_actions = []
                action_values = []
                if maze[i,j-1] > 0
                    possible_actions = [possible_actions, "north"]
                    action_values    = [action_values, values[maze[i,j-1]]]
                end
                if maze[i,j+1] > 0
                    possible_actions = [possible_actions, "south"]
                    action_values    = [action_values, values[maze[i,j+1]]]
                end

                if maze[i-1,j] > 0
                    possible_actions = [possible_actions, "west"]
                    action_values    = [action_values, values[maze[i-1,j]]]
                end
                if maze[i+1,j] > 0
                    possible_actions = [possible_actions, "east"]
                    action_values    = [action_values, values[maze[i+1,j]]]
                end
                best_action_value = maximum(action_values)
                possible_actions = possible_actions[findin(action_values,best_action_value)]
                action = possible_actions[int(ceil(rand() * length(possible_actions)))]
                m.policy[maze[i,j]] = action
            end
        end
    end
end


function create_maze_element(maze::String, state_values::Dict{Any, Any})
    string_states = convert_string_to_string_array(maze)
    named_states = name_states(string_states)
    values = initialise_value_function(named_states)
    m = Maze(maze, string_states, named_states, values, [], state_values)
    greedy_policy!(m)
    m
end


function update_value_function!(m::Maze)
    m
end

update_value_function! (generic function with 1 method)

In [12]:
maze = create_maze_element(maze_def, state_values)
#maze_state_names = name_states(maze_array)

Maze("\n#######\n#T    #\n#A    #\n#     #\n#     #\n#    T#\n#######\n",7x7 Array{Char,2}:
 '#'  '#'  '#'  '#'  '#'  '#'  '#'
 '#'  'T'  'A'  ' '  ' '  ' '  '#'
 '#'  ' '  ' '  ' '  ' '  ' '  '#'
 '#'  ' '  ' '  ' '  ' '  ' '  '#'
 '#'  ' '  ' '  ' '  ' '  ' '  '#'
 '#'  ' '  ' '  ' '  ' '  'T'  '#'
 '#'  '#'  '#'  '#'  '#'  '#'  '#',7x7 Array{Int64,2}:
 0   0   0   0   0   0  0
 0  -1  -1   1   2   3  0
 0   4   5   6   7   8  0
 0   9  10  11  12  13  0
 0  14  15  16  17  18  0
 0  19  20  21  22  -1  0
 0   0   0   0   0   0  0,[0,0,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0],String["east","south","north","south","east","east","south","west","east","east"  …  "north","west","north","west","west","north","west","south","north","north"],{' '=>-1,'T'=>0})

In [13]:
greedy_policy!(maze)